# Welcome to the OmniScript Prompt Editor Tool
2023 05 23

&nbsp;  

This tool is designed to make it easier for you to work with and refine your ChatGPT prompts.

This notebook is not designed to be run from beginning to end in its entirety (although you can certainly do that, especially when you first start it up).

&nbsp;  

---

&nbsp;  

## Imports / Setup
The next block is used to bring in (and in the case of openai, install) the libraries we need to run the tool.  Each time you spin up ("connect") to a colab engine, you'll need to do this.

In [ ]:
# Make sure we have openai
!pip install openai

# The libraries we need:
from jinja2 import Template
import json
import openai
import textwrap

## Inputs
There are three inputs we need to perform the test:

1. Our OpenAI API Key
1. The Data JSON from our OmniScript at the point we're ready to call ChatGPT
1. The "System Prompt" which defines what we want ChatGPT to do for us
1. The template we use to convert the data into the "User Prompt"

The next four blocks set those up.  It is expected that you'll play with the last two blocks (the system prompt and the user template) over and over again, which is why they are last.

In [ ]:
#
#   To do the testing of our prompts, we will need an OpenAI API Key.
#   These are kind of valuable, so you don't want to let them leak out.
#   On the other hand, constantly setting them is a pain as well.
#   
#   So, I offer two choices (but there are many, many more) for you.
#   
#   Choice #1: Ask the user for the key each time this block is run

OPENAI_API_KEY = input('Please enter your OpenAI API Key: ')

#   Choice #2, just embed the key in the notebook.  You're editing
#   you're own local copy, not mine, so its safe to do this.  If
#   you want to save your key, go to File→Save a copy in Drive
#   (You have to have a Google account for this)

# OPENAI_API_KEY = 'XYZZY'

#   Choose whichever you prefer!

In [ ]:
#
#   Paste your Data JSON here between the """s
#

data_json = """
{"timeStamp":"2023-05-22T19:23:12.668Z","userProfile":"System Administrator","userTimeZoneName":"America/Los_Angeles","userTimeZone":"-420","userCurrencyCode":"USD","userName":"charles-ost@mcguinness.us","userId":"005Dp000001WbTWIA0","Step1":{"selWhite":"1","selBkack":"0","selRed":"1","selBlue":"1","selGreen":"1","selCyan":"1","selMagenta":"0","selYellow":"1"}}
"""

In [ ]:
#
#   Build / Edit your system prompt here
#
system_prompt = """
You are an order management specialist at a widget factory.  
There are 8 different types of widgets: 

* The primary colors: red, blue, and green
* The secondary colors: yellow, magenta, and cyan
* Two desaturated colors: black, and white.

For every cyan widget someone buys, they must also buy an addtional blue and 
green widget as well.

For every yellow widget, they must also buy an additional red and green widget.

For every magenta widget, they must also buy an additional red and a blue widget.

The widgets in red, blue, green, black, and white have no restrictions on them.

Thus, if you buy two secondary colored widgets, you have to buy the 
corresponding four primary colored widgets to meet the requirements.

For example, if you buy yellow and magenta, you will also have to have two
reds, a green, and a blue in your order.

Your job is to look at an order a customer wants to make and to verify that it 
follows the rules.

Please explain, step by step, how you evaluate the rules.
"""

In [ ]:
#
#   Build / Edit your template here:
#
template_content = """
I would like to buy widgets in the following colors and quantities:

White: {{ Step1.selWhite }}
Black: {{ Step1.selBlack }}
Red: {{ Step1.selRed }}
Blue: {{ Step1.selBlue }}
Green: {{ Step1.selGreen }}
Cyan: {{ Step1.selCyan }}
Magenta: {{ Step1.selMagenta }}
Yellow: {{ Step1.selYellow }}

Is that allowed?
"""


# Excecution
There are two blocks that follow:

1. A block that creates our prompt from our data and template
2. A block that calls ChatGPT with that

You can run the first one over and over again until you think the prompt is right and then run the second to actually transmit it to ChatGPT for a live test.

In [ ]:
template = Template(template_content)
user_prompt = template.render(json.loads(data_json))
print(user_prompt)

In [ ]:
# Execute this block if you want to send everything to ChatGPT:

openai.api_key = OPENAI_API_KEY

completion = openai.ChatCompletion.create(
    # model="gpt-3.5-turbo",
    model="gpt-4",
    temperature=0,
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ]
)

# Unfortunately, we need to word-wrap the output to read it.  This
# code's kind of ugly and partially stolen from stackoverflow, but it works.
lines = (completion.choices[0].message["content"]).split("\n")
lists = (textwrap.TextWrapper(width=90,break_long_words=False).wrap(line) for line in lines)
body  = "\n".join("\n".join(list) for list in lists)
print(body)
